# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

Still being developed, but you may get started on this when you are finished with Assignment 3.

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
	* [Initial Code](#Initial-Code)
	* [Add moves counter](#Add-moves-counter)
	* [negamaxIDS](#negamaxIDS)
	* [negamaxIDSab](#negamaxIDSab)
	* [Grading](#Grading)
	* [Extra Credit](#Extra-Credit)


For this assignment, you will investigate the advantages of alpha-beta
pruning applied to Tic-Tac-Toe.  To do so, follow these steps.

## Initial Code <font color='red'>UPDATED Oct 8</font>

In [1]:
def negamax(game, depthLeft):
    import copy
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
 
    for move in game.getMoves():
        
        game.makeMove(move)
       
        value, _ = negamax(game, depthLeft-1)
        
        game.unmakeMove(move)
        
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

In [2]:
class TTT(object):

    def __init__(self):
        self.board = [' ']*9
        self.player = 'X'
        if False:
            self.board = ['X', 'X', ' ', 'X', 'O', 'O', ' ', ' ', ' ']
            self.player = 'X'
        self.playerLookAHead = self.player
        self.movesExplored=0

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
        self.movesExplored+=1

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
     
        
    def getNumberMovesExplored(self):
        return self.movesExplored
    
    def initialPlayerWin(self,score):
        if score==1:
            return True
        else:
            return False
        
    def update(self, board, lookA):
        for i in range(len(board)):
            self.board[i]=board[i]
        self.playerLookAHead=lookA

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s

Check that the following function `playGame` runs
correctly. Notice that we are using *negamax* to find the best move for
Player X, but Player O, the opponent, is using function *opponent*
that follows the silly strategy of playing in the first open position.

In [3]:
def opponent(board):
    import random
    listActions= [i for i, mark in enumerate(board) if mark == ' ']
    random.shuffle(listActions)
    return listActions[0]

Modifying the opponent function, the oponent player will choose a ramdom position to play within the possible options

In [4]:
def playGame(game,opponent,depthLimit):
    import random
    print(game)
    while not game.isOver():
       # print 'board to make move', game.board
        score,move = negamaxIDSab(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        #print 'board to make move1', game.board
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()

Having  the opponent player choosing a ramdom position to play instead of the next available possition, the game is always won by the player that is using a search algorithm to determine wich is the best move to do. 

## Add moves counter

Evaluate the efficiency of the search by keepting track of the number of nodes explored, which is the same as the number of moves explored, during a game. Do this by adding a counter named `movesExplored` to the `TTT` constructor where it is initialized to 0 and increment the counter in the `TTT.makeMove` method.  Add a method `ttt.getNumberMovesExplored()` to get its current value.  So

    print('Number of moves explored', game.getMovesExplored())
    
will print the number of moves explored. You will not use a global variable for counting this time.

## negamaxIDS 

<font color='red'>UPDATED Oct 4</font>

Write a new function named `negamaxIDS` that performs an iterative deepening negamax search.  Replace the first line in the `while` loop of `playGame` with

        score,move = negamaxIDS(game,depthLimit)
        
where `depthLimit` is now the maximum depth and multiple `negamax` searches are performed for depth limits of $1, 2, \ldots,$ maximum depth.

But, when should you stop?  Can you stop before readhing the depthLimit?  If not, there is no point to doing iterative deepening.

For Tic-Tac-Toe, we can stop as soon as a call to `negamax` returns a winning move.  This will have a value of 1 for Tic-Tac-Toe.  To keep our `negamaxIDS` function general, add a method called `getWinningValue` to the `TTT` class that just returns 1.  Then `negamaxIDS` can call `game.getWinningValue()` to determine the value of a winning move for this game.  If the maximum depth is reached and no winning move has been found, return the best move found over all depth limts.

In [5]:
def negamaxIDS(game,maxDepth):
    for x in range(maxDepth+1):
        score, move = negamax(game,x)
        if initialPlayerWin(score)==True:
            break
    return score, move

In [6]:
def initialPlayerWin(score):
    if score==1:
        return True
    else:
        return False
    

## negamaxIDSab

Now for the hardest part.  Make a new function `negamaxIDSab` by duplicating `negamaxIDS` and add the code to implement alpha-beta pruning.

The algorithm negamaxIDSab is a derivation of negamaxIDS but it has a better performance. It does not explores the branches whose utility is not benefitial for the current player, because there is another previous move that is better than the one trying to be explored.

In [7]:
def negamaxIDSab(game,maxDepth):
    for x in range(maxDepth+1):
        score,move= negamaxab(game,x,-float('infinity'), float('infinity'))
        if initialPlayerWin(score)==True:
            break
    return score,move

In [8]:
def negamaxab(game, depthLeft, alpha, betha):
    if game.isOver() or depthLeft == 0:
        return game.getUtility(),None
    bestValue= -float('infinity')
    bestMove=None

    for move in game.getMoves():
        game.makeMove(move)
        value,_ = negamaxab(game, depthLeft-1, -betha, -alpha)
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        bestValue = max(bestValue,value)
        bestMove=move
        alpha = max(alpha,value)
        #Prunning the rest of the branches
        if alpha >= betha:
            break
    return bestValue, bestMove


## playGames

Now duplicate the game playing loop so three complete tic-tac-toe games are played.  Call this new version `playGames`. For the first game, use `negamax`. For the second game, use `negamaxIDS`.  For the third game, use `negamaxIDSab`.  At the end of each game, print the number of X's in the final board, the number moves explored, the depth of the game which is the number of moves made by X and O, and the effective branching factor.  When you run `playGames` you should see the tic-tac-toe positions after each move and, after all games are done, a line for each game like the following lines, which were <font color='red'>UPDATED Oct 8</font>.

    negamax made 4 moves. 558334 moves explored for ebf(558334, 7) of 6.46
    negamaxIDS made 3 moves. 23338 moves explored for ebf(23338, 5) of 7.26
    negamaxIDSab made 3 moves 6053 moves explored for ebf(6053, 5) of 5.48

Your results may be different. 

The value of the depth is the total number of moves made by X and by O during the search.  You can calculate this by keeping a list of all board states, or by just counting the number of X's and O's in the final board.

In [9]:
def ebf(nNodes, depth, precision=0.01):
    if nNodes == 0:
        return 0

    def ebfRec(low, high):
        mid = (low + high) * 0.5
        if mid == 1:
            estimate = 1 + depth
        else:
            estimate = (1 - mid**(depth + 1)) / (1 - mid)
        if abs(estimate - nNodes) < precision:
            return mid
        if estimate > nNodes:
            return ebfRec(low, mid)
        else:
            return ebfRec(mid, high)

    return ebfRec(1, nNodes)

The previous algorithm was provided by Dr. Chuck Anderson in order to perform the effective branching factor

In [10]:
def playGames(gameOriginal,opponent,depthLimit):
    import copy
    results=[]
    movesEx=[]
    for searchAlgorithm in [negamax, negamaxIDS, negamaxIDSab]:     
        game=copy.deepcopy(gameOriginal)
        print 'Algorithm: ',searchAlgorithm.__name__
        print(game)
        while not game.isOver():
            
            score,move = searchAlgorithm(game,depthLimit)
            if move == None :
                print('move is None. Stopping.')
                break
            game.makeMove(move)
            print('Player', game.player, 'to', move, 'for score' ,score)
            print(game)
            if not game.isOver():
                game.changePlayer()
                opponentMove = opponent(game.board)
                game.makeMove(opponentMove)
                print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
                print(game)
                game.changePlayer()
            
        results.append(game.movesExplored)
        nMoves=[i for i, mark in enumerate(game.board) if mark != ' ']
        movesEx.append(len(nMoves))
        
    print ('negamax made'  'moves.' ,results[0],' moves explored for ebf(',results[0],',' 
           ,movesEx[0],')of',ebf(results[0],movesEx[0]),')')
    print ('negamaxIDS made'  'moves.' ,results[1],' moves explored for ebf(',results[1],',' 
           ,movesEx[1],')of',ebf(results[1],movesEx[1]),')')
    print ('negamaxIDSab made'  'moves.' ,results[2],' moves explored for ebf(',results[2],',' 
           ,movesEx[2],')of',ebf(results[2],movesEx[2]),')')

    
    print results
    

Here are some example results. <font color='red'>Updated October 8, 3:15pm </font>

In [11]:
game=TTT()
playGames(game,opponent,7)

Algorithm:  negamax
 | | 
-----
 | | 
-----
 | | 
('Player', 'X', 'to', 0, 'for score', 1)
X| | 
-----
 | | 
-----
 | | 
('Player', 'O', 'to', 4)
X| | 
-----
 |O| 
-----
 | | 
('Player', 'X', 'to', 1, 'for score', 0)
X|X| 
-----
 |O| 
-----
 | | 
('Player', 'O', 'to', 7)
X|X| 
-----
 |O| 
-----
 |O| 
('Player', 'X', 'to', 2, 'for score', 1)
X|X|X
-----
 |O| 
-----
 |O| 
Algorithm:  negamaxIDS
 | | 
-----
 | | 
-----
 | | 
('Player', 'X', 'to', 0, 'for score', 1)
X| | 
-----
 | | 
-----
 | | 
('Player', 'O', 'to', 6)
X| | 
-----
 | | 
-----
O| | 
('Player', 'X', 'to', 1, 'for score', 1)
X|X| 
-----
 | | 
-----
O| | 
('Player', 'O', 'to', 5)
X|X| 
-----
 | |O
-----
O| | 
('Player', 'X', 'to', 2, 'for score', 1)
X|X|X
-----
 | |O
-----
O| | 
Algorithm:  negamaxIDSab
 | | 
-----
 | | 
-----
 | | 
('Player', 'X', 'to', 8, 'for score', -1)
 | | 
-----
 | | 
-----
 | |X
('Player', 'O', 'to', 0)
O| | 
-----
 | | 
-----
 | |X
('Player', 'X', 'to', 7, 'for score', 1)
O| | 
-----
 | | 
-----
 |X|

In the previous example its easy to see that the most effective algorithm is the negamaxIDSab because was able to find a solution exploring less nodes than the other algorithms.


## Grading

As always, download and extract from [A4grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A4grader.tar)

In [12]:
%run -i A4grader.py

('\nTesting negamax starting from', ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' '])
('\n--- 10/10 points. negamax correctly returns value of', 1)
('\n--- 10/10 points. negamax correctly explored', 124, 'states.')
('\nTesting negamax starting from', ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' '])
('\n--- 10/10 points. negamax correctly returns value of', -1, 'and move of', 5)
('\nTesting negamaxIDS with max depth of 5, starting from', ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' '])
('\n--- 10/10 points. negamaxIDS correctly returns value of', -1, 'and move of', 5)
('\nTesting negamaxIDSab starting from', ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' '])
('\n--- 20/20 points. negamaxIDSab correctly returns value of', -1, 'and move of', 8)

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
('Player', 'X', 'to', 0, 'for score', 0)
X| | 
-----
 | | 
-----
 | | 
('Player', 'O', 'to', 8)
X| | 
-----
 | | 
-----
 | |O
('Player', 'X', 'to', 2

## Extra Credit -Othello Game

Earn one extra credit point for each of the following.

  - Implement another game and repeat the above steps.

  - Implement a random move chooser as the opponent (Player O) and determine how many times Player X can win against this opponent as an average over multiple games.

For the extra credit the adversarial search was implemented to solve the Othello game. It is an example of how the previous algorithms can be used to show which is the best move for the initial player in order to end the game with the greatest number of tiles.

The algorithms to do the search where modified and adapted to the Othello game in order to unmake a move,that is a little bit different to unmake a move in Tic-tac-toe, because in Othello one move includes modifying several tiles at the same time, and there is no way to keep track of which tile was modified other than storing the state board after calling the recursive function and restablished that value after the recursive function has ended.

In [13]:
def negamaxO(game, depthLeft):
    import copy
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    #used to unmake move to Othello function
    tiles=[]
    for c in game.board:
        tiles.append(c)
    for move in game.getMoves():
        
        game.makeMove(move)
       
        value, _ = negamaxO(game, depthLeft-1)
        
        game.unmakeMove(move,tiles)
        
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

In [14]:
def negamaxIDSO(game,maxDepth):
    for x in range(maxDepth+1):
        score, move = negamaxO(game,x)
        if initialPlayerWin(score)==True:
            break
    return score, move

In [15]:
def negamaxIDSabO(game,maxDepth):
    for x in range(maxDepth+1):
        score,move= negamaxabO(game,x,-float('infinity'), float('infinity'))
        if initialPlayerWin(score)==True:
            break
    return score,move

In [16]:
def negamaxabO(game, depthLeft, alpha, betha):
    if game.isOver() or depthLeft == 0:
        return game.getUtility(),None
    bestValue= -float('infinity')
    bestMove=None
    tiles=[]
    for c in game.board:
        tiles.append(c)
    for move in game.getMoves():
        game.makeMove(move)
        value,_ = negamaxabO(game, depthLeft-1, -betha, -alpha)
        game.unmakeMove(move,tiles)
        if value is None:
            continue
        value = - value
        bestValue = max(bestValue,value)
        bestMove=move
        alpha = max(alpha,value)
        #Prunning the rest of the branches
        if alpha >= betha:
            break
    return bestValue, bestMove


In [17]:
class Othello(object):

    def __init__(self):
        self.board = [' ']*16
        #initial positions
        self.board[5]='X'
        self.board[6]='O'
        self.board[9]='O'
        self.board[10]='X'
        #initial player
        self.player = 'X'
        
        
        if False:
            self.board = ['O']*16
            #initial positions
            self.board[5]='O'
            self.board[6]='O'
            self.board[9]='O'
            self.board[10]='X'
            self.board[1]='O'
            self.board[0]='X'

            self.player = 'X'
        self.playerLookAHead = self.player
        self.movesExplored=0

      
    
    
 
            
    def getMoves(self):
        
        moves=[]
        move=self.locations(' ')
        oponent='O' if self.playerLookAHead == 'X' else 'X'  
        dim=4
        tilesToFlip=0
        directions=[[0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1]]
        #for each blanck space evaluate all directions to see if a valid move
        for row,column in move:
            #print 'row',row, 'col', column
            emptyRow,emptyColumn=row,column

            for rowDirection,columnDirection in directions:
                row=emptyRow
                column=emptyColumn
                #print 'directio',rowDirection,columnDirection 
                #print 'columnDirection',columnDirection
                row+=rowDirection
                column+=columnDirection
                if  not self.insideBoard(row,column):
                    continue
                tilesToFlip=0
                while self.board[row*dim+column]==oponent:
                    tilesToFlip+=1
                    #print 'oponent'
                    row+=rowDirection
                    column+=columnDirection             
                    if not self.insideBoard(row,column):
                        break
                if not self.insideBoard(row,column):
                        continue
                if self.board[row*dim+column]==self.playerLookAHead and tilesToFlip>0 :
                    #print 'player'
                    moves.append((emptyRow,emptyColumn))
                    break 
        return moves

     
       
 
        
    def insideBoard(self,row,column):
        return  row >= 0 and row <= 3 and column >= 0 and column <=3
    
    #def isValidMove(self,move):
        
    
    def makeMove(self, move):
        self.tilesToFlip=[]
        oponent='O' if self.playerLookAHead == 'X' else 'X'  
        dim=4
        tilesToFlip=0
        directions=[[0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1]]
        #for each blanck space evaluate all directions to see if a valid move
        #for row,column in move:
        #print 'row',row, 'col', column
        if move!=[]:
             
            #print ' emptyRow,emptyColumn',  move
            #print 'move', move
            emptyRow,emptyColumn=move
        
            for rowDirection,columnDirection in directions:
                row=emptyRow
                column=emptyColumn
                #print 'directio',rowDirection,columnDirection 
                #print 'columnDirection',columnDirection
                row+=rowDirection
                column+=columnDirection
                if  not self.insideBoard(row,column):
                    continue
                tilesToFlip=0
                while self.board[row*dim+column]==oponent:
                    
                    tilesToFlip+=1
                    #print 'oponent'
                    row+=rowDirection
                    column+=columnDirection             
                    if not self.insideBoard(row,column):
                        break
                if not self.insideBoard(row,column):
                        continue
                if self.board[row*dim+column]==self.playerLookAHead and tilesToFlip>0 :
                    #print 'player'
                    stepsBack=tilesToFlip
                    while stepsBack >= 0:
                        #print 'oponent'
                        row-=rowDirection
                        column-=columnDirection
                        self.board[row*dim+column] = self.playerLookAHead
                        self.tilesToFlip.append(row*dim+column)
                        stepsBack-=1

                        #moves.append(((emptyRow,emptyColumn),(rowDirection,columnDirection)))
                        #break 
            self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
            self.movesExplored+=1

    def locations(self, c):
        dim=4
        moves=[]
        spaces= [i for i, mark in enumerate(self.board) if mark == c]
        for space in spaces:
            x=space/dim
            y=space%dim
            moves.append((x,y))   
        return moves
    
    def isOver(self):
        #print 'is over'
        moves=self.getMoves()
        return True if len(moves) <=0 else False
        #return self.getMoves is []
    
    def getUtility(self):
        
        avalMoves=self.getMoves()
        #print 'moves avalable', avalMoves
        numberX = len(self.locations('X'))
        numberO = len(self.locations('O'))
        winner= 'X' if numberX > numberO else 'O'
        if  numberX > numberO and ' ' not in self.board:
            return 1 if self.playerLookAHead is 'X' else -1
        elif numberO > numberX and ' ' not in self.board:
            #print 'o win', self.playerLookAHead
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else :   
            if len(avalMoves)<0:
                if  numberX > numberO :
                    return 1 if self.playerLookAHead is 'X' else -1
                elif numberO > numberX:
                    return 1 if self.playerLookAHead is 'O' else -1
            else:              
                return None

        
        
        
    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move,tiles):
        import copy
        for c in range(len(tiles)-1):   
            self.board[c]=tiles[c]
        #print 'board', self.board
        #print 'copy',tiles
     
       
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
        
        
    def getNumberMovesExplored(self):
        return self.movesExplored
    
    def initialPlayerWin(self,score):
        if score==1:
            return True
        else:
            return False

    

    def __str__(self):
        s = '{}|{}|{}|{}\n-------\n{}|{}|{}|{}\n-------\n{}|{}|{}|{}\n-------\n{}|{}|{}|{}'.format(*self.board)
        return s

In [18]:
def playGamesO(gameOriginal,opponent,depthLimit):
    import copy
    results=[]
    movesEx=[]
    for searchAlgorithm in [negamaxO, negamaxIDSO, negamaxIDSabO]:     
        game=copy.deepcopy(gameOriginal)
        print 'Algorithm: ',searchAlgorithm.__name__
        print(game)
        while not game.isOver():
            
            score,move = searchAlgorithm(game,depthLimit)
            if move == None :
                print('move is None. Stopping.')
                break
            game.makeMove(move)
            print('Player', game.player, 'to', move, 'for score' ,score)
            print(game)
            if not game.isOver():
                game.changePlayer()
                mvs=game.getMoves()
                opponentMove = mvs[0]
                game.makeMove(opponentMove)
                print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
                print(game)
                game.changePlayer()
            
        results.append(game.movesExplored)
        nMoves=[i for i, mark in enumerate(game.board) if mark != ' ']
        movesEx.append(len(nMoves))
        
    print ('negamax made'  'moves.' ,results[0],' moves explored for ebf(',results[0],',' 
           ,movesEx[0],')of',ebf(results[0],movesEx[0]),')')
    print ('negamaxIDS made'  'moves.' ,results[1],' moves explored for ebf(',results[1],',' 
           ,movesEx[1],')of',ebf(results[1],movesEx[1]),')')
    print ('negamaxIDSab made'  'moves.' ,results[2],' moves explored for ebf(',results[2],',' 
           ,movesEx[2],')of',ebf(results[2],movesEx[2]),')')

    
    print results
    

In [19]:
game = Othello()
playGamesO(game,opponent ,12)

Algorithm:  negamaxO
 | | | 
-------
 |X|O| 
-------
 |O|X| 
-------
 | | | 
('Player', 'X', 'to', (0, 2), 'for score', 0)
 | |X| 
-------
 |X|X| 
-------
 |O|X| 
-------
 | | |X
('Player', 'O', 'to', (0, 1))
 |O|X| 
-------
 |O|X| 
-------
 |O|X| 
-------
 | | |X
('Player', 'X', 'to', (0, 0), 'for score', 1)
X|X|X| 
-------
 |X|X| 
-------
 |O|X| 
-------
 | | |X
('Player', 'O', 'to', (0, 3))
X|X|X|O
-------
 |X|O| 
-------
 |O|X| 
-------
 | | |X
('Player', 'X', 'to', (1, 3), 'for score', 1)
X|X|X|O
-------
 |X|X|X
-------
 |O|X| 
-------
 | | |X
('Player', 'O', 'to', (2, 3))
X|X|X|O
-------
 |X|X|O
-------
 |O|O|O
-------
 | | |X
('Player', 'X', 'to', (3, 0), 'for score', 1)
X|X|X|O
-------
 |X|X|O
-------
 |X|O|O
-------
X| | |X
('Player', 'O', 'to', (1, 0))
X|X|X|O
-------
O|O|O|O
-------
 |X|O|O
-------
X| | |X
('Player', 'X', 'to', (3, 2), 'for score', 1)
X|X|X|O
-------
O|O|X|O
-------
 |X|X|O
-------
X| |X|X
('Player', 'O', 'to', (2, 0))
X|X|X|O
-------
O|O|X|O
-------
O|O|O|O